In [135]:
import requests
import os
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO

import pandas as pd

## Creating functions

In [132]:
def create_soup(url):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    # Send an HTTP request to the URL
    response = requests.get(url, headers = header)
    soup = BeautifulSoup(response.content, 'html.parser')

    return soup


In [140]:
def images_to_blob(image, image_names):
    print(image_names)

In [142]:
def data_to_SQL(prod_name, prod_description,brand,prod_images_names):
    print(prod_name)

In [143]:
def get_image_links(prod_soup):
    
    item_list_1 = prod_soup.select('div div div div div ul li button div img')

    ##### get image
    images = []
    i = 0
    for item in item_list_1: 
            
        if i%2 == 0:
            #print(item['src'])
            if (i == 2) | (i == 8) | (i == 10):
                images = images + [item['src']]
        
        i += 1
    
    return images

In [144]:
def get_description(prod_soup):
    
    description_list = prod_soup.select('dd')

    #Get description
    description = ''
    for d in description_list:
        description = description + ' ' + d.text

    #Get Materials
    description_list = prod_soup.find_all(class_ = 'f8f91e f02838' )

    description = description + ' Materials: '

    for d in description_list[0:2]:
        description = description + ' ' + d.text

    return description
    

In [ ]:
def links_to_images(links,prod_name):
    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    i=0
    for img_link in links:
        response = requests.get(img_link, headers = header)
        img = Image.open(BytesIO(response.content))

        # Resize the image to 600
        img_resized = img.resize((500, 500))

        img_name = f"HM{prod_name}_{i}.jpg"

        # Load to Blob
        images_to_blob(img, img_name)
        i+=1



In [148]:
def HM(category_url,n_products):

    soup = create_soup(category_url)
    item_list = soup.select('li section article div div div div ul li a')


    br = 0
    for item in item_list: 
            
        if br == n_products:
            break
        else:

            #Scrapp
            prod_soup = create_soup(item['href'])

            prod_images_links = get_image_links(prod_soup=prod_soup)
            prod_description = get_description(prod_soup=prod_soup)

            prod_name = 'HM_'+str(br)  
            
            # Links to image, load to blob and return prod_images_names
            prod_images_names = links_to_images(links = prod_images_links,prod_name= br)
            
            data_to_SQL(prod_name=prod_name, prod_description=prod_description,brand='HM',prod_images_names=prod_images_names)

        br += 1


In [149]:
# Define the URL you want to scrape
url = 'https://www2.hm.com/it_it/donna/acquista-per-prodotto/top.html'

HM(url,3)

Gucci0_11.jpg
Gucci0_11.jpg
Gucci0_11.jpg
HM_0
Gucci1_11.jpg
Gucci1_11.jpg
Gucci1_11.jpg
HM_1
Gucci2_11.jpg
Gucci2_11.jpg
Gucci2_11.jpg
HM_2
